In [ ]:
"""
LINCS 2020数据加载和预处理脚本 - DrugReflector论文严格复现版
适配 Expanded CMap LINCS Resource 2020 数据集
完全按照Science 2025补充材料的预处理流程实现
优化版：提升运行效率，使用向量化操作

数据来源：Expanded CMap LINCS Resource 2020
- Dataset: CMAP LINCS 2020
- Updated: 11/23/2021
- Source: https://clue.io/data/CMap2020#LINCS2020
- Level 4 (Z-scores): level4_beta_trt_cp_n1805898x12328.gctx
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')


class LINCS2020DataLoader:
    """
    加载和预处理LINCS 2020数据 - 严格遵循DrugReflector论文流程
    
    数据集信息：
    - 基因信息: geneinfo_beta.txt (1.09 MB)
    - 细胞信息: cellinfo_beta.txt (0.04 MB)  
    - 化合物信息: compoundinfo_beta.txt (4.42 MB)
    - Level 4数据: level4_beta_trt_cp_n1805898x12328.gctx (82.94 GB)
    """
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.signatures = None
        self.decompressed_files = []
        
        print(f"\n{'='*80}")
        print(f"🔬 LINCS 2020 Data Loader Initialized")
        print(f"{'='*80}")
        print(f"Data directory: {self.data_dir}")
        print(f"Dataset: Expanded CMap LINCS Resource 2020")
        print(f"{'='*80}\n")
    
    def load_gene_info(self):
        """
        加载基因信息，获取978个landmark genes
        
        文件: geneinfo_beta.txt
        关键列:
        - pr_gene_id: 基因ID
        - pr_gene_symbol: 基因符号
        - pr_is_lm: landmark基因标记 (1表示landmark)
        - pr_is_bing: bing基因标记
        """
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(
                f"❌ Gene info file not found: {gene_file}\n"
                f"   Please download 'geneinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"📖 Loading gene information...")
        print(f"   File: {gene_file.name}")
        
        # LINCS 2020使用制表符分隔
        gene_info = pd.read_csv(gene_file, sep='\t')
        
        print(f"   ✓ Loaded {len(gene_info):,} genes")
        
        # 筛选landmark genes (pr_is_lm == 1)
        if 'pr_is_lm' in gene_info.columns:
            landmark_genes = gene_info[gene_info['pr_is_lm'] == 1].copy()
        elif 'feature_space' in gene_info.columns:
            # 备用字段
            landmark_genes = gene_info[gene_info['feature_space'] == 'landmark'].copy()
        else:
            raise ValueError(
                "Cannot identify landmark genes. "
                "Expected 'pr_is_lm' or 'feature_space' column."
            )
        
        print(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        print(f"   ✓ Expected: 978")
        
        if len(landmark_genes) != 978:
            print(f"   ⚠️  Warning: Found {len(landmark_genes)} landmark genes, expected 978")
        
        self.gene_info = landmark_genes
        return landmark_genes
    
    def load_cell_info(self):
        """
        加载细胞系信息
        
        文件: cellinfo_beta.txt
        关键列:
        - cell_id: 细胞系ID
        - cell_lineage: 细胞谱系
        - primary_disease: 主要疾病
        """
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(
                f"❌ Cell info file not found: {cell_file}\n"
                f"   Please download 'cellinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"\n📖 Loading cell information...")
        print(f"   File: {cell_file.name}")
        
        cell_info = pd.read_csv(cell_file, sep='\t')
        
        print(f"   ✓ Loaded {len(cell_info):,} cell lines")
        
        if 'cell_id' in cell_info.columns:
            print(f"   ✓ Unique cell IDs: {cell_info['cell_id'].nunique()}")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """
        加载化合物信息
        
        文件: compoundinfo_beta.txt
        关键列:
        - pert_id: 扰动ID (BRD-XXXXXXXXX)
        - pert_iname: 化合物名称
        - pert_type: 扰动类型
        - canonical_smiles: SMILES结构
        """
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            print(f"⚠️  Compound info file not found: {compound_file}")
            print(f"   This file is optional but recommended.")
            return None
        
        print(f"\n📖 Loading compound information...")
        print(f"   File: {compound_file.name}")
        
        compound_info = pd.read_csv(compound_file, sep='\t')
        
        print(f"   ✓ Loaded {len(compound_info):,} compounds")
        
        if 'pert_id' in compound_info.columns:
            print(f"   ✓ Unique perturbagens: {compound_info['pert_id'].nunique()}")
        
        # 统计化合物类型
        if 'pert_type' in compound_info.columns:
            print(f"\n   Perturbagen types:")
            for ptype, count in compound_info['pert_type'].value_counts().head(5).items():
                print(f"     - {ptype}: {count:,}")
        
        self.compound_info = compound_info
        return compound_info
    
    def decompress_gctx_file(self, gctx_file):
        """
        如果GCTX文件被压缩，则解压
        注意：LINCS 2020的GCTX文件通常不压缩（已经是HDF5二进制格式）
        """
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        # 如果是.gz文件，需要解压
        if str(gctx_file).endswith('.gz'):
            print(f"⚠️  Detected compressed GCTX file: {gctx_file.name}")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                print(f"✓ Found existing decompressed file: {output_path.name}")
                return str(output_path)
            
            print(f"📦 Decompressing...")
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            print(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """
        读取GCTX文件 (HDF5格式)
        
        LINCS 2020 GCTX结构：
        - /0/DATA/0/matrix: 数据矩阵 (samples × genes)
        - /0/META/ROW: 样本元数据
        - /0/META/COL: 基因元数据
        
        Parameters:
        -----------
        gctx_file: str or Path
            GCTX文件路径
        use_landmark_only: bool
            是否只保留landmark genes
        """
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        # 检查文件大小
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        print(f"   File size: {file_size_gb:.2f} GB")
        
        if file_size_gb > 50:
            print(f"   ⚠️  Large file detected. This may take 10-30 minutes...")
        
        # 解压（如果需要）
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Loading matrix from HDF5...")
            
            # LINCS 2020使用标准GCTX v1.0结构
            if '/0/DATA/0/matrix' in f:
                matrix = f['/0/DATA/0/matrix'][:]
            elif '/matrix' in f:
                matrix = f['/matrix'][:]
            else:
                raise ValueError(f"Cannot find matrix in GCTX file. Available keys: {list(f.keys())}")
            
            print(f"✓ Matrix shape: {matrix.shape} (samples × genes)")
            
            print(f"📋 Loading metadata...")
            
            # 读取样本元数据（ROW）
            sample_meta = {}
            row_path = '/0/META/ROW' if '/0/META/ROW' in f else '/row'
            
            for key in f[row_path].keys():
                data = f[f'{row_path}/{key}'][:]
                if data.dtype.kind in ['S', 'O', 'U']:
                    # 字符串类型
                    try:
                        sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                    except:
                        sample_meta[key] = data.astype(str)
                else:
                    sample_meta[key] = data
            
            # 读取基因元数据（COL）
            gene_meta = {}
            col_path = '/0/META/COL' if '/0/META/COL' in f else '/col'
            
            for key in f[col_path].keys():
                data = f[f'{col_path}/{key}'][:]
                if data.dtype.kind in ['S', 'O', 'U']:
                    try:
                        gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                    except:
                        gene_meta[key] = data.astype(str)
                else:
                    gene_meta[key] = data
        
        sample_df = pd.DataFrame(sample_meta)
        gene_df = pd.DataFrame(gene_meta)
        
        print(f"\n✓ Data loaded successfully:")
        print(f"  Matrix: {matrix.shape} (samples × genes)")
        print(f"  Samples: {len(sample_df):,}")
        print(f"  Genes: {len(gene_df)}")
        
        # 过滤landmark genes
        if use_landmark_only:
            print(f"\n🔬 Filtering to landmark genes...")
            
            if self.gene_info is None:
                print(f"   Loading gene info...")
                self.load_gene_info()
            
            landmark_ids = set(self.gene_info['pr_gene_id'].astype(str).values)
            print(f"   Landmark genes to find: {len(landmark_ids)}")
            
            # LINCS 2020中基因ID可能在'id'或'pr_gene_id'列
            if 'id' in gene_df.columns:
                gene_id_col = 'id'
            elif 'pr_gene_id' in gene_df.columns:
                gene_id_col = 'pr_gene_id'
            else:
                raise ValueError(f"Cannot find gene ID column. Available: {gene_df.columns.tolist()}")
            
            gene_mask = gene_df[gene_id_col].astype(str).isin(landmark_ids)
            n_matched = gene_mask.sum()
            print(f"   ✓ Matched: {n_matched} genes")
            
            if n_matched == 0:
                raise ValueError("No landmark genes matched!")
            
            print(f"\n🎯 Applying filter...")
            matrix = matrix[:, gene_mask]
            gene_df = gene_df[gene_mask].reset_index(drop=True)
            
            print(f"   ✓ Final matrix shape: {matrix.shape}")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """
        加载Level 4数据 (Z-scores)
        
        文件: level4_beta_trt_cp_n1805898x12328.gctx
        - trt: treatment (处理)
        - cp: compound perturbation (化合物扰动)
        - n1805898: 180万+样本
        - x12328: 12,328个基因 (978 landmark + 推断基因)
        """
        # 查找Level 4化合物处理文件
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            # 尝试通配符匹配
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            
            if not files:
                raise FileNotFoundError(
                    f"❌ Level 4 file not found: {level4_file}\n"
                    f"   Please download 'level4_beta_trt_cp_n1805898x12328.gctx' from:\n"
                    f"   https://clue.io/data/CMap2020#LINCS2020\n"
                    f"   File size: 82.94 GB"
                )
            
            level4_file = files[0]
        
        print(f"\n{'='*80}")
        print(f"📖 Loading Level 4 Signatures")
        print(f"{'='*80}")
        print(f"File: {level4_file.name}")
        
        # 读取GCTX（只保留landmark genes）
        matrix, sample_meta, gene_meta = self.read_gctx(
            level4_file, 
            use_landmark_only=True
        )
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        return matrix, sample_meta, gene_meta
    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series
    ) -> np.ndarray:
        """
        计算每个样本与其最近replicate的余弦相似度（向量化优化）
        
        原文方法（SI第2页）：
        "For each compound, remove any observations with a cosine similarity <0.12 
        to the closest replicate"
        
        Parameters:
        -----------
        matrix: np.ndarray
            表达矩阵 (n_samples, n_genes)
        pert_ids: pd.Series
            每个样本的化合物ID
        
        Returns:
        --------
        np.ndarray: 每个样本与其最近replicate的余弦相似度
        """
        print(f"\n📊 Calculating cosine similarity to nearest replicate...")
        n_samples = len(pert_ids)
        nearest_similarities = np.zeros(n_samples)
        
        unique_perts = pert_ids.unique()
        print(f"   Processing {len(unique_perts):,} compounds...")
        
        # 进度条
        from tqdm import tqdm
        
        for pert_id in tqdm(unique_perts, desc="   Computing similarities"):
            pert_mask = pert_ids == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            if len(pert_indices) < 2:
                # 只有一个样本，设为0（会被过滤）
                nearest_similarities[pert_indices] = 0.0
                continue
            
            pert_data = matrix[pert_mask]
            
            # 使用sklearn快速计算成对余弦相似度
            sim_matrix = cosine_similarity(pert_data)
            
            # 将对角线设为-inf，避免样本与自己比较
            np.fill_diagonal(sim_matrix, -np.inf)
            
            # 对每个样本找到最大相似度（最近的replicate）
            max_sims = np.max(sim_matrix, axis=1)
            
            nearest_similarities[pert_indices] = max_sims
        
        print(f"   ✓ Calculated similarities for {n_samples:,} samples")
        print(f"   Mean similarity: {nearest_similarities.mean():.4f}")
        print(f"   Median similarity: {np.median(nearest_similarities):.4f}")
        print(f"   Min similarity: {nearest_similarities.min():.4f}")
        print(f"   Max similarity: {nearest_similarities.max():.4f}")
        
        return nearest_similarities
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=5,      # 原文filter 2
        min_replicate_similarity=0.12,        # 原文filter 3
        dose_range=(1.0, 20.0),               # 原文filter 4: 1-20µM
        valid_timepoints=['6 h', '24 h'],     # 原文filter 5 (LINCS 2020格式)
        min_cell_lines=5,                     # 原文filter 6
        max_cell_lines=40,                    # 原文filter 6
        remove_dos=True                       # 原文filter 1
    ):
        """
        准备训练数据，严格按照论文SI第2页的过滤流程
        
        原文过滤顺序：
        1. Remove DOS compounds
        2. Remove compounds with <5 observations
        3. Remove observations with cosine similarity <0.12 to closest replicate
        4. Select most frequent dose between 1-20µM for each compound
        5. Keep only 6h or 24h measurements
        6. Remove compounds in <5 or >40 cell lines
        
        Parameters:
        -----------
        min_observations_per_compound: int
            每个化合物的最小观测数 (原文: 5)
        min_replicate_similarity: float
            与最近replicate的最小余弦相似度 (原文: 0.12)
        dose_range: tuple
            有效剂量范围 µM (原文: 1-20)
        valid_timepoints: list
            有效时间点 (原文: 6h, 24h)
            注意: LINCS 2020使用 '6 h', '24 h' 格式
        min_cell_lines: int
            每个化合物的最小细胞系数 (原文: 5)
        max_cell_lines: int
            每个化合物的最大细胞系数 (原文: 40)
        remove_dos: bool
            是否移除DOS化合物 (原文: True)
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE")
        print("   Following Science 2025 Supplementary Materials (Page 2)")
        print("   Dataset: LINCS 2020")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        
        # 打印可用的元数据列
        print(f"\n📋 Available metadata columns:")
        for col in row_meta.columns[:20]:  # 显示前20列
            print(f"   - {col}")
        if len(row_meta.columns) > 20:
            print(f"   ... and {len(row_meta.columns) - 20} more")
        
        # LINCS 2020关键字段映射
        # 常见字段名: pert_id, cell_id, pert_time, pert_dose, pert_type
        
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()
        initial_compounds = working_meta['pert_id'].nunique() if 'pert_id' in working_meta.columns else 0
        
        print(f"\nInitial compounds: {initial_compounds:,}")
        
        # ========== Filter 1: Remove DOS compounds ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS (Diversity-Oriented Synthesis) compounds")
            print(f"{'='*80}")
            
            # LINCS 2020中，pert_type字段标识化合物类型
            if 'pert_type' in working_meta.columns:
                # 保留 'trt_cp' (化合物处理)，移除DOS等其他类型
                dos_mask = working_meta['pert_type'] == 'trt_cp'
                n_dos = (~dos_mask).sum()
            else:
                # 通过pert_id判断（BRD-DOS开头或包含DOS）
                dos_mask = ~working_meta['pert_id'].str.contains('DOS', case=False, na=False)
                n_dos = (~dos_mask).sum()
            
            working_matrix = working_matrix[dos_mask]
            working_meta = working_meta[dos_mask].reset_index(drop=True)
            
            print(f"  Removed {n_dos:,} DOS observations")
            print(f"  Remaining samples: {len(working_meta):,}")
            print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations per compound ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"{'='*80}")
        
        obs_counts = working_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = working_meta['pert_id'].isin(valid_perts)
        working_matrix = working_matrix[obs_mask]
        working_meta = working_meta[obs_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 3: Cosine similarity to closest replicate ==========
        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"         to closest replicate")
        print(f"{'='*80}")
        
        # 计算每个样本与其最近replicate的余弦相似度
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            working_matrix, 
            working_meta['pert_id']
        )
        
        # 过滤低相似度样本
        sim_mask = nearest_similarities >= min_replicate_similarity
        n_removed_sim = (~sim_mask).sum()
        
        working_matrix = working_matrix[sim_mask]
        working_meta = working_meta[sim_mask].reset_index(drop=True)
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 4: Dose selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Select most frequent dose in range {dose_range[0]}-{dose_range[1]} µM")
        print(f"{'='*80}")
        
        if 'pert_dose' in working_meta.columns:
            # LINCS 2020剂量格式: "10 um", "1 um"等
            # 解析剂量值和单位
            working_meta['dose_value'] = pd.to_numeric(
                working_meta['pert_dose'].astype(str).str.extract(r'(\d+\.?\d*)')[0], 
                errors='coerce'
            )
            
            # 解析剂量单位
            working_meta['dose_unit'] = working_meta['pert_dose'].astype(str).str.extract(r'([a-zA-Z]+)')[0]
            
            # 转换为µM
            def convert_to_uM(row):
                if pd.isna(row['dose_value']):
                    return np.nan
                value = row['dose_value']
                unit = str(row['dose_unit']).lower() if pd.notna(row['dose_unit']) else 'um'
                
                if 'nm' in unit:
                    return value / 1000  # nM to µM
                elif 'mm' in unit:
                    return value * 1000  # mM to µM
                else:  # assume µM
                    return value
            
            working_meta['dose_uM'] = working_meta.apply(convert_to_uM, axis=1)
            
            # 筛选有效剂量范围
            dose_mask = (
                (working_meta['dose_uM'] >= dose_range[0]) & 
                (working_meta['dose_uM'] <= dose_range[1])
            )
            
            print(f"  Samples in valid dose range: {dose_mask.sum():,}")
            
            # 对每个化合物，选择最常见的剂量
            valid_doses = []
            for pert_id in working_meta['pert_id'].unique():
                pert_mask = (working_meta['pert_id'] == pert_id) & dose_mask
                if pert_mask.sum() == 0:
                    continue
                
                # 找出最常见的剂量
                dose_counts = working_meta.loc[pert_mask, 'dose_uM'].value_counts()
                if len(dose_counts) > 0:
                    most_common_dose = dose_counts.index[0]
                    valid_doses.append(
                        (working_meta['pert_id'] == pert_id) & 
                        (working_meta['dose_uM'] == most_common_dose)
                    )
            
            if valid_doses:
                dose_final_mask = pd.concat([pd.Series(mask) for mask in valid_doses], axis=1).any(axis=1)
                n_removed_dose = (~dose_final_mask).sum()
                
                working_matrix = working_matrix[dose_final_mask]
                working_meta = working_meta[dose_final_mask].reset_index(drop=True)
                
                print(f"  Removed {n_removed_dose:,} observations (invalid or non-modal dose)")
                print(f"  Remaining samples: {len(working_meta):,}")
                print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
            else:
                print(f"  ⚠️  Warning: No valid doses found in range, skipping dose filter")
        else:
            print(f"  ⚠️  Warning: 'pert_dose' column not found, skipping dose filter")
        
        # ========== Filter 5: Timepoint selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints}")
        print(f"{'='*80}")
        
        if 'pert_time' in working_meta.columns:
            # LINCS 2020时间格式: "6 h", "24 h"
            # 标准化时间点格式
            working_meta['time_normalized'] = working_meta['pert_time'].astype(str).str.lower().str.strip()
            
            # 规范化valid_timepoints
            normalized_valid_times = []
            for t in valid_timepoints:
                normalized_valid_times.extend([
                    t.lower().strip(),
                    t.lower().strip().replace(' ', ''),
                    t.lower().strip().replace('h', ' h'),
                    t.lower().strip().replace('h', 'hr')
                ])
            
            time_mask = working_meta['time_normalized'].isin(normalized_valid_times)
            
            if time_mask.sum() == 0:
                # 如果没有匹配，显示可用的时间点
                print(f"  ⚠️  Warning: No samples match timepoints {valid_timepoints}")
                print(f"  Available timepoints:")
                for tp, count in working_meta['pert_time'].value_counts().head(10).items():
                    print(f"    - {tp}: {count:,} samples")
                print(f"  Skipping timepoint filter...")
            else:
                n_removed_time = (~time_mask).sum()
                working_matrix = working_matrix[time_mask]
                working_meta = working_meta[time_mask].reset_index(drop=True)
                
                print(f"  Removed {n_removed_time:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {len(working_meta):,}")
                print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        else:
            print(f"  ⚠️  Warning: 'pert_time' column not found, skipping timepoint filter")
        
        # ========== Filter 6: Cell line count per compound ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"{'='*80}")
        
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        valid_perts_cell = cell_line_counts[
            (cell_line_counts >= min_cell_lines) & 
            (cell_line_counts <= max_cell_lines)
        ].index
        
        print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
              f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
        
        cell_mask = working_meta['pert_id'].isin(valid_perts_cell)
        working_matrix = working_matrix[cell_mask]
        working_meta = working_meta[cell_mask].reset_index(drop=True)
        
        print(f"  Removed {(~cell_mask).sum():,} observations")
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Create final dataset ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET (After all filters)")
        print(f"{'='*80}")
        
        # 创建化合物标签编码
        unique_perts = sorted(working_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in working_meta['pert_id']])
        
        final_compounds = len(unique_perts)
        final_samples = len(working_matrix)
        final_cells = working_meta['cell_id'].nunique()
        
        print(f"  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {working_matrix.shape[1]}")
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): "
              f"{working_meta.groupby('pert_id').size().median():.0f}")
        
        # 统计化合物分布
        compound_obs = working_meta.groupby('pert_id').size()
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        # 与原文对比
        print(f"\n📊 Comparison with paper results:")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        training_data = {
            'X': working_matrix,
            'y': labels,
            'folds': np.zeros(len(working_matrix), dtype=int),  # 临时值，稍后会被create_3fold_splits覆盖
            'sample_meta': working_meta,
            'metadata': working_meta,  # 添加此行 - 兼容训练代码
            'gene_names': list(col_meta['id'].values if 'id' in col_meta.columns else col_meta['pr_gene_id'].values),  # 转为list
            'compound_names': list(unique_perts),  # 确保是list
            'pert_to_idx': pert_to_idx
        }
                
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """
        创建3折交叉验证划分
        
        原文（SI第2页）：
        "The training data was divided randomly into three folds, 
        with perturbation replicates balanced across the folds."
        """
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=int)
        
        # 按化合物分配fold，确保同一化合物的所有样本在同一fold
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            # 随机打乱
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            # 尽可能均匀分配到3个fold
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


# ========== 主程序 ==========

def main():
    """
    主程序 - 严格按照DrugReflector论文流程
    使用LINCS 2020数据集
    """
    print("=" * 80)
    print("🧬 DRUGREFLECTOR DATA PREPROCESSING")
    print("   Dataset: Expanded CMap LINCS Resource 2020")
    print("   Method: Science 2025 Supplementary Materials")
    print("=" * 80)
    
    # 初始化加载器
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    loader = LINCS2020DataLoader(data_dir)
    
    try:
        # Step 1: 加载元数据
        print("\n" + "=" * 80)
        print("STEP 1: Loading metadata")
        print("=" * 80)
        gene_info = loader.load_gene_info()
        cell_info = loader.load_cell_info()
        compound_info = loader.load_compound_info()
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures (Z-scores)")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures()
        
        # Step 3: 准备训练数据（严格按照论文流程）
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data (Paper-compliant pipeline)")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=5,    # Filter 2
            min_replicate_similarity=0.12,      # Filter 3
            dose_range=(1.0, 20.0),             # Filter 4
            valid_timepoints=['6 h', '24 h',
                              '6h', '24h',
                              '6hr', '24hr'
                              '6 hr, 24 hr'],   # Filter 5
            min_cell_lines=5,                   # Filter 6
            max_cell_lines=40,                  # Filter 6
            remove_dos=True                     # Filter 1
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存处理后的数据
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True, parents=True)
        
        output_file = output_dir / "training_data_lincs2020_paper_compliant.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        print(f"   File size: {output_file.stat().st_size / (1024**2):.1f} MB")
        
        # 打印最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Data shape: {training_data['X'].shape}")
        print(f"   • Average samples per compound: "
              f"{len(training_data['X']) / len(training_data['compound_names']):.1f}")
        
        print(f"\n🎯 Next steps:")
        print(f"   1. Run training: python drugreflector_training.py")
        print(f"   2. Or use quick start: python quick_train.py")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        print(f"\n📋 Full traceback:")
        import traceback
        traceback.print_exc()
        
        print(f"\n💡 Troubleshooting tips:")
        print(f"   1. Check if all LINCS 2020 files are downloaded")
        print(f"   2. Verify file paths and names")
        print(f"   3. Ensure sufficient disk space (>100 GB)")
        print(f"   4. Check RAM availability (>32 GB recommended)")
        
        return None


if __name__ == "__main__":
    training_data = main()

🧬 DRUGREFLECTOR DATA PREPROCESSING
   Dataset: Expanded CMap LINCS Resource 2020
   Method: Science 2025 Supplementary Materials

🔬 LINCS 2020 Data Loader Initialized
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Dataset: Expanded CMap LINCS Resource 2020


STEP 1: Loading metadata
📖 Loading gene information...
   File: geneinfo_beta.txt
   ✓ Loaded 12,328 genes
   ✓ Landmark genes: 978
   ✓ Expected: 978

📖 Loading cell information...
   File: cellinfo_beta.txt
   ✓ Loaded 240 cell lines

📖 Loading compound information...
   File: compoundinfo_beta.txt
   ✓ Loaded 39,321 compounds
   ✓ Unique perturbagens: 34419

STEP 2: Loading Level 4 signatures (Z-scores)

📖 Loading Level 4 Signatures
File: level4_beta_trt_cp_n1805898x12328.gctx

📖 Reading GCTX file: level4_beta_trt_cp_n1805898x12328.gctx
   File size: 82.94 GB
   ⚠️  Large file detected. This may take 10-30 minutes...
📊 Loading matrix from HDF5...
